In [ ]:
%load_ext autoreload
%autoreload 2

import os
os.environ['OPENAI_API_KEY'] = 'XXX'


import pandas as pd
from vjp.openai_prompt import Prompt
import vjp.data as data
import xml.etree.ElementTree as ET
from functools import partial

In [ ]:
def clean_list_et(documents: list[ET.Element]):
    for doc in documents:
        doc: ET.Element = doc
        assert len(list(doc.iter('intro'))) <= 1, doc.attrib

   
    for doc in documents:
        doc: ET.Element = doc
        parent_map = {c: p for p in doc.iter() for c in p}
        intros = list(doc.iter('intro'))
        for i in intros:
            parent_map[i].remove(i)

In [ ]:
upheld, rejected = data.load_second_instance_labeled()
clean_list_et(upheld)
clean_list_et(rejected)
df = data.shot_based_dataframe(upheld, rejected)

In [ ]:
prompt = Prompt()

In [ ]:
def compute_new_column(row, with_mot):
    message = prompt.create_prompt(row, with_mot)
    response = prompt.send_prompt(message)
    try:
        msg = prompt.interpret_response(response)
        return msg
    except:
        print(prompt.get_nice_prompt(message))
        print(response)
        

compute_new_column_no_mot = partial(compute_new_column, with_mot=False)
compute_new_column_mot = partial(compute_new_column, with_mot=True)

df['pred_label_no_mot'] = df.apply(compute_new_column_no_mot, axis=1)
#df['pred_label_mot'] = df.apply(compute_new_column_mot, axis=1)

In [ ]:
df.to_pickle("dataframe_no_mot_2")

In [ ]:
import pickle

with open("backup_cache_no_mot_2", 'wb') as f:
    pickle.dump(prompt.cache, f)

In [ ]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(df['label'], df['pred_label_no_mot']))

In [ ]:
df

In [ ]:
for _, row in df.iterrows():
    message = prompt.create_prompt(row)
    print(prompt.get_nice_prompt(message))
    
    if row['label'] == 1:
        true = "uphold"
    else:
        true = "reject"

    if row['pred_label_no_mot'] == 1:
        pred = "uphold"
    else:
        pred = "reject"

    print(f"True: {true} - Predicted: {pred}")